In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3195801 sha256=b6753d478ead5886e4e41e36c69cab2c1a40ada8aaa459f87a18424b8c0dfe31
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from surprise.model_selection import train_test_split
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.accuracy import mae
from surprise.accuracy import mse
from surprise.accuracy import rmse
from surprise import KNNBasic

from google.colab import drive
drive.mount('/gdrive', force_remount=True)
drive.mount('/content/drive')

Mounted at /gdrive
Mounted at /content/drive


In [ ]:
#loading the dataset
df = pd.read_csv('/content/drive/My Drive/MLProject/train_cleaned.csv')
df

,Unnamed: 0,product_id,brand,price,user_id,purchase_or_cart,cat_0,cat_1,ts_weekday,ts_day,ts_month
0,0,1005014,samsung,503.09,533326659,0,electronics,smartphone,4,1,11
1,1,1005115,apple,949.47,565865924,0,electronics,smartphone,4,1,11
2,2,1002542,apple,486.80,549256216,0,electronics,smartphone,4,1,11
3,3,1002542,apple,486.80,549256216,0,electronics,smartphone,4,1,11
4,4,4804056,apple,160.57,522355747,0,electronics,audio,4,1,11
...,...,...,...,...,...,...,...,...,...,...,...
9312755,11495236,26300086,lucente,237.84,514851004,1,appliances,kitchen,4,31,1
9312756,11495238,1004210,samsung,79.54,571761801,1,construction,tools,4,31,1
9312757,11495239,5100562,apple,294.34,518849571,1,electronics,clocks,4,31,1
9312758,11495240,1005186,samsung,720.46,599883250,1,appliances,kitchen,4,31,1


In [ ]:
#dropping unnecessary columns and duplicates
df.drop([ 'ts_weekday','ts_month','ts_day',], axis=1, inplace=True)
df.drop(df.columns[[0]],axis = 1, inplace = True)
df.drop_duplicates(inplace = True)
df

,product_id,brand,price,user_id,purchase_or_cart,cat_0,cat_1
0,1005014,samsung,503.09,533326659,0,electronics,smartphone
1,1005115,apple,949.47,565865924,0,electronics,smartphone
2,1002542,apple,486.80,549256216,0,electronics,smartphone
4,4804056,apple,160.57,522355747,0,electronics,audio
5,1004873,samsung,362.29,563558500,0,electronics,smartphone
...,...,...,...,...,...,...,...
9312752,1005212,samsung,164.06,519212350,1,construction,tools
9312753,1004767,samsung,226.18,608821433,1,construction,tools
9312754,1005184,samsung,977.86,595018267,1,appliances,kitchen
9312755,26300086,lucente,237.84,514851004,1,appliances,kitchen


In [ ]:
df_new= []
df_new= df['user_id'].unique()
df_new=np.sort(df_new)

In [ ]:
#generating a dataframe for 10000 users with all user interactions
df.sort_values(by = ['user_id'])
temp_df=df[(df['user_id'] <  df_new[15000])]

In [ ]:
temp_df

,product_id,brand,price,user_id,purchase_or_cart,cat_0,cat_1
54,1801766,artel,146.85,508622143,0,electronics,video
125,1004739,xiaomi,203.09,512411198,0,electronics,smartphone
644,2900569,lg,90.17,512387304,0,appliances,kitchen
726,1005100,samsung,139.68,512381448,0,electronics,smartphone
727,1801766,artel,146.85,512389812,0,electronics,video
...,...,...,...,...,...,...,...
9311569,100086203,samsung,125.36,512416542,1,construction,tools
9311600,1005264,xiaomi,155.71,512416542,1,construction,tools
9312249,1005115,apple,806.61,512429826,1,construction,tools
9312253,37000007,x-game,170.50,512380433,1,appliances,environment


In [ ]:
reader = Reader(rating_scale=(0, 1))

dataset = Dataset.load_from_df(temp_df[['user_id', 'product_id', 'purchase_or_cart']], reader)

In [ ]:
#building train and test set.
trainset, testset = train_test_split(dataset, test_size=0.2, random_state=42)

In [ ]:
from collections import defaultdict

def get_top_n(uid,pid, n):
  # Get recommendations for a user
  user_id =uid # Replace with the actual user_id for which you want to get recommendations
  n_recommendations = n  # Number of recommendations to get
  product_id = pid

  # Get items that the user has not seen
  user_unseen_items = temp_df.loc[~temp_df['product_id'].isin(temp_df.loc[temp_df['user_id'] == user_id, 'product_id']), 'product_id'].unique()

  # Get similarity for unseen items
  user_unseen_item_similarity = [(item_id, algo_knn.predict(user_id, item_id).est) for item_id in user_unseen_items]

  # Sort items based on similarity in descending order
  user_unseen_item_similarity.sort(key=lambda x: x[1], reverse=True)

  # Get the top n recommendations
  recommended_item_ids = [item[0] for item in user_unseen_item_similarity[:n_recommendations]]

  # Get details of recommended items
  recommended_items = temp_df[temp_df['product_id'].isin(recommended_item_ids)][['product_id', 'brand', 'price','cat_0','cat_1']]
  print("Recommended items for user_id", user_id)
  print(recommended_items[:n])

  

In [ ]:
#training model
algo_knn = KNNBasic(k = 20)
algo_knn.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
#getting all user interactions for a specific user
uid = 512411198
pid = 1004739
temp_df.loc[(temp_df['user_id'] == uid)]


,product_id,brand,price,user_id,purchase_or_cart,cat_0,cat_1
125,1004739,xiaomi,203.09,512411198,0,electronics,smartphone
1794,1004741,xiaomi,189.97,512411198,0,electronics,smartphone
80434,1005161,xiaomi,208.10,512411198,1,electronics,smartphone
1884922,1004767,samsung,254.82,512411198,0,electronics,smartphone


In [ ]:
#generating predictions for the user 
get_top_n(uid, pid, 10)

Recommended items for user_id 512411198
       product_id     brand   price        cat_0       cat_1
2733     10900055  scarlett   11.81   appliances     kitchen
4466      1004887      oppo  128.68  electronics  smartphone
6742      1004887      oppo  128.68  electronics  smartphone
8562     10900055  scarlett   11.81   appliances     kitchen
12359     1005101    xiaomi  431.40  electronics  smartphone
16090     1002629     apple  358.31  electronics  smartphone
16558     1004781    huawei  254.65  electronics  smartphone
16809     1004834   samsung  172.18  electronics  smartphone
24742     1004781    huawei  254.63  electronics  smartphone
28547     4804718     apple  360.09  electronics       audio


In [ ]:
#calculating RMSE 
predictions = algo_knn.test(testset)
rmse(predictions, verbose= True)
mse(predictions, verbose = True)
mae(predictions, verbose = True)

RMSE: 0.5322
MSE: 0.2832
MAE:  0.4518


0.4517930522294208